# Caribbean fish hotspots

This is code for Southeast Conservation Blueprint 2023. Written by Rua Mordecai.

## Import libraries and define variables

In [95]:
import os
import arcpy

In [96]:
# define spatial reference, workspaces, and source data location
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\CaribbeanBlueprint2023_FinalIndicators\CaribbeanBlueprint2023_FinalIndicators\SpatialData"
benthicPR = r"D:\CaribbeanData\TNC\BenthicCondition\BENTHIC_CONDITION_PRVI\pr_mar_benhab4m_2021_EEZs_gridcode.tif"
benthicVI = r"D:\CaribbeanData\TNC\BenthicCondition\BENTHIC_CONDITION_PRVI\usvi_mar_benhab4m_2021_EEZs_gridcode.tif"
CCAPPR = r"D:\Landcover\CCAP\Caribbean\pr_2010_ccap_hr_land_cover20170214.img"
CCAPStCroix = r"D:\Landcover\CCAP\Caribbean\usvi_stcroix_2012_ccap_hr_land_cover20150910.img"
CCAPStJohn = r"D:\Landcover\CCAP\Caribbean\usvi_stjohn_2012_ccap_hr_land_cover20140915.img"
CCAPStThomas = r"D:\Landcover\CCAP\Caribbean\usvi_stthomas_2012_ccap_hr_land_cover20150914.img"

In [97]:
# define final output name
IndicatorFileName = "CaribbeanFishHotspots.tif"

In [98]:
# define raster used for cell size, extent, and snapping
CaribbeanRaster= r"D:\Blueprint\2023\extent\VIPR_Extent_v6.tif"

In [99]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

## Start analysis

### Prep data

In [100]:
# Set the source workspace
arcpy.env.workspace = OutputWorkspace

In [70]:
# Combine TNC benthic data
arcpy.management.MosaicToNewRaster(benthicPR+";"+benthicVI, OutputWorkspace, "vipr_mar_benhab4m_2021_EEZs_gridcode.tif", None, "8_BIT_UNSIGNED", None, 1, "LAST", "FIRST")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\WaterscapeHabitat\\working\\vipr_mar_benhab4m_2021_EEZs_gridcode.tif'>

In [71]:
# Combine CCAP data
arcpy.management.MosaicToNewRaster(CCAPPR+";"+CCAPStCroix+";"+CCAPStJohn+";"+CCAPStThomas, OutputWorkspace, "vipr_ccap.tif", None, "8_BIT_UNSIGNED", None, 1, "LAST", "FIRST")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\WaterscapeHabitat\\working\\vipr_ccap.tif'>

In [72]:
# reproject benthic data and convert to 30 meters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
        arcpy.management.Resample("vipr_mar_benhab4m_2021_EEZs_gridcode.tif", "benthic.tif", "30 30", "MAJORITY")

In [73]:
# reproject ccap data and convert to 30 meters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
        arcpy.management.Resample("vipr_ccap.tif", "vipr_ccap30.tif", "30 30", "MAJORITY")

In [74]:
# reclassify so dense seagrass is 1 and everything else is 0. Make NoData 1000 so we can combine with CCAP mangrove data.
out_raster = arcpy.sa.Reclassify("benthic.tif", "Value", "1 11 0;12 0;13 1;NoData 1000", "DATA"); out_raster.save("Seagrass.tif")

In [75]:
# reclassify so Estuarine forested wetland is 1 and everything else is 0. This is to map mangrove. Make NoData 0 to combine with seagrass.
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify("vipr_ccap30.tif", "Value", "1 15 0;16 1;17 0;18 0;19 23 0;NoData 0", "DATA"); out_raster.save("Mangrove.tif")

In [76]:
# reclassify so coral is 1
out_raster = arcpy.sa.Reclassify("benthic.tif", "Value", "1 5 1;6 13 0;NoData 0", "DATA"); out_raster.save("Coral.tif")

In [77]:
# Combine rasters to address overlapping pixel predictions and differences in NoData between CCAP and TNC benthic
out_raster = arcpy.sa.Plus("Seagrass.tif", "Mangrove.tif"); out_raster.save("SeagrassMangrove.tif")

In [78]:
# Make a mask to remove land (including mangrove) based on combined seagrass and mangrove layer
out_raster = arcpy.sa.Reclassify("SeagrassMangrove.tif", "Value", "0 999 1;1000 NoData;1001 NoData", "DATA"); out_raster.save("waterMask.tif")

In [79]:
# Reclassify seagrass to convert NoData back to 0
out_raster = arcpy.sa.Reclassify("Seagrass.tif", "Value", "0 0;1 1;1000 0", "DATA"); out_raster.save("SeagrassBin.tif")

### 600m radius analysis

In [80]:
# 20 cell max for 600m radius. These ID whether there's at least one pixel of the habitat in 600m radius  
out_raster = arcpy.ia.FocalStatistics("Mangrove.tif", "Circle 20 CELL", "MAXIMUM", "DATA", 90); out_raster.save("MangroveBin20.tif")

In [81]:
# 20 cell max for 600m radius.  
out_raster = arcpy.ia.FocalStatistics("Coral.tif", "Circle 20 CELL", "MAXIMUM", "DATA", 90); out_raster.save("CoralBin20.tif")

In [82]:
# 20 cell max for 600m radius.  
out_raster = arcpy.ia.FocalStatistics("SeagrassBin.tif", "Circle 20 CELL", "MAXIMUM", "DATA", 90); out_raster.save("SeagrassBin20.tif")

In [83]:
# Sum habitat diversity then multiply by mangrove to remove places without any mangrove in 600m radius
output_raster = arcpy.ia.RasterCalculator(["CoralBin20.tif","MangroveBin20.tif","SeagrassBin20.tif"],['A','B','C'],'(A + B + C)*10'); output_raster.save("habRichness600m.tif")

### 300m radius analysis

In [84]:
# 10 cell max for 300m radius. These ID whether there's at least one pixel of the habitat in 300m radius  
out_raster = arcpy.ia.FocalStatistics("Mangrove.tif", "Circle 10 CELL", "MAXIMUM", "DATA", 90); out_raster.save("MangroveBin10.tif")

In [85]:
# 10 cell max for 300m radius.  
out_raster = arcpy.ia.FocalStatistics("Coral.tif", "Circle 10 CELL", "MAXIMUM", "DATA", 90); out_raster.save("CoralBin10.tif")

In [86]:
# 10 cell max for 300m radius.  
out_raster = arcpy.ia.FocalStatistics("SeagrassBin.tif", "Circle 10 CELL", "MAXIMUM", "DATA", 90); out_raster.save("SeagrassBin10.tif")

In [87]:
# Combine habitats info to get number of habitats
output_raster = arcpy.ia.RasterCalculator(["CoralBin10.tif","MangroveBin10.tif","SeagrassBin10.tif"],['A','B','C'],'A + B + C'); output_raster.save("habRichness300m.tif")

### Combine multiscale analysis

In [88]:
# Combine to results from both scales
out_raster = arcpy.sa.Plus("habRichness600m.tif", "habRichness300m.tif"); out_raster.save("multiScaleHab.tif")

In [89]:
# Reclassify for final indicator categories
out_raster = arcpy.sa.Reclassify("multiScaleHab.tif", "Value", "0 0;10 0;11 0;20 1;21 1;22 3;30 2;31 2;32 3;33 4", "DATA"); out_raster.save("multiScaleHabClass.tif")

In [90]:
# Remove land (including mangrove) from indicator
out_raster = arcpy.sa.Times("multiScaleHabClass.tif", "waterMask.tif"); out_raster.save("multiScaleHabClassNoLand.tif")

In [91]:
# clip raster to Caribbean extent
out_raster = arcpy.sa.ExtractByMask("multiScaleHabClassNoLand.tif", CaribbeanRaster); out_raster.save(IndicatorFileName)

## Finalize indicator

In [94]:
# set code block for legend
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = Low predicted fish density/diversity (no coral, mangrove, or dense seagrass present within 600 m of one other)'
    elif value == 1:
        return '1 = Medium predicted fish density/diversity (either mangrove and coral, mangrove and dense seagrass, or coral and dense seagrass present within 600 m)'    
    elif value == 2:
        return '2 = High predicted fish density/diversity (mangrove, coral, and dense seagrass all present within 600 m'
    elif value == 3:
        return '3 = Very high predicted fish density/diversity (either mangrove and coral, mangrove and dense seagrass, or coral and dense seagrass present within 300 m)' 
    elif value == 4:
        return '4 = Highest predicted fish density/diversity (mangrove, coral, and dense seagrass all present within 300 m)'             
"""

In [95]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\WaterscapeHabitat\\working\\CaribbeanFishHotspots.tif'>

In [101]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 4:
		return 169
	if Value == 3:
		return 186
	if Value == 2:
		return 187
	if Value == 1:
		return 178
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 4:
		return 49
	if Value == 3:
		return 78
	if Value == 2:
		return 117
	if Value == 1:
		return 158
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 4:
		return 84
	if Value == 3:
		return 146
	if Value == 2:
		return 190
	if Value == 1:
		return 218
	if Value == 0:
		return 255
	else:
		return 255
		
"""

In [102]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanFishHotspots.tif'>